# Paso 1. Preparando todo






## 1.1. Vamos a instalar pysentimiento
https://www.kaggle.com/datasets/kavita5/twitter-dataset-avengersendgame?resource=download

In [ ]:
!pip install pysentimiento

## 1.2. Ahora vamos a importar create_analyzer, eligiendo "sentiment" para sentimiento y el idioma

In [ ]:
from pysentimiento import create_analyzer
analyzer = create_analyzer(task="sentiment", lang="en")

## 1.3. Test de create_analyzer

In [ ]:
analyzer.predict("What a time to be alive")
# returns AnalyzerOutput(output=POS, probas={POS: 0.973, NEU: 0.025, NEG: 0.002})

In [ ]:
analyzer.predict("This shit can't be real")
# returns AnalyzerOutput(output=NEG, probas={NEG: 0.937, NEU: 0.055, POS: 0.008})

In [ ]:
analyzer.predict("You piss me off")
# returns AnalyzerOutput(output=NEG, probas={NEG: 0.981, NEU: 0.013, POS: 0.006})

In [ ]:
analyzer.predict("jejeje I don't trust you")
# returns AnalyzerOutput(output=NEG, probas={NEG: 0.971, NEU: 0.026, POS: 0.004})

# Paso 2. Google Drive

## 2.1. Montando el drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Paso 3. Trabajando con el csv de tweets

## 3.1. Para descargar el csv con todos los tweets acceder al siguiente enlace:
https://www.kaggle.com/datasets/kavita5/twitter-dataset-avengersendgame?resource=download

## 3.2. Importar el contenido del csv a un DataFrame

In [ ]:
import pandas as pd

file = pd.read_csv('/content/drive/MyDrive/Python_final_module/ex00/tweets.csv', encoding='ISO-8859-1')

## 3.3. Mostrar el contenido del DataFrame

In [ ]:
file

## 3.4. Mostrar una columna del DataFrame


In [ ]:
file['text']

## 3.5. Guardar los tweets del df en una lista

In [ ]:
lst = []
for value in file['text'].iteritems():
  lst.append(value[1])
print(lst)

# Paso 4. Cálculos a partir de los tweets

## 4.1. Filtrado de los tweets

In [ ]:
filtered_lst = []
for tweet in lst:
  split_lst = tweet.split(' ')
  new_lst = []
  for word in split_lst:
    if word != 'RT' and not word.startswith('@') \
    and not '#' in word and not word.startswith('https')\
    and not word.startswith('<'):
      new_lst.append(word)
  filtered_lst.append(' '.join(new_lst))


In [ ]:
#     ======= Si no lo has hecho ya =======
# from pysentimiento import create_analyzer
#    analyzer = create_analyzer(task="sentiment", lang="en")

## 4.2. Analizando los tweets

4.2.1. Pasamos la lista de tweets por el analyzer

In [ ]:
analyzed_lst = []
for tweet in filtered_lst:
  analyzed_lst.append(analyzer.predict(tweet))
print(analyzed_lst)


### 4.2.2. Totales de los outputs

In [ ]:
def getFromOutput():
  neg = 0
  pos = 0
  neu = 0
  total = 0 
  for item in analyzed_lst:
    string = str(item)
    if "output=NEU" in string:
      neu += 1
    if "output=NEG" in string:
      neg += 1
    if "output=POS" in string:
      pos += 1
    total += 1

  return {"total" : total, "neg" : neg / total * 100, \
          "pos" :pos / total * 100, "neu" : neu / total * 100}
  # print("Total: {}, Neg: {:.2f}%, Pos: {:.2f}%, and Neu: {:.2f}%".\
  #   format(total, neg / total * 100, pos / total * 100, neu / total * 100))

print(getFromOutput())

### 4.2.3. Totales de los resultados de las pruebas

In [ ]:
def chopped_results():
  neg = 0
  pos = 0
  neu = 0
  total = 0
  new_lst = []
  for item in analyzed_lst:
    mini_list = str(item).split('probas={')[1].replace('})', '').split(', ')
    new_lst.append(mini_list)
  for result in new_lst:
    for item in result:
      result = str(item).split(' ')
      if "NEU" in result[0]:
        neu += float(result[1])
      if "NEG" in result[0]:
        neg += float(result[1])
      if "POS" in result[0]:
        pos += float(result[1])
    total += 1
  return {"total" : total, "neg" : neg / total * 100, \
          "pos" :pos / total * 100, "neu" : neu / total * 100}
  # print("Total: {}, Neg: {:.2f}%, Pos: {:.2f}%, and Neu: {:.2f}%".\
  #   format(total, neg / total * 100, pos / total * 100, neu / total * 100))

print(chopped_results())


### 4.2.4. Gráfica a partir del resultado

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Ejemplo de DataFrame con datos en porcentajes
results = chopped_results()
df = pd.DataFrame({'Sentiment': ['Neg', 'Pos', 'Neu'],
    'Percentage': [results['neg'], results['pos'], results['neu']]})

# Crear el gráfico de pastel
ax = df.plot(kind='pie', y='Percentage', labels=df['Sentiment'],
    autopct='%1.1f%%', legend=None)

# Agregar título al gráfico
ax.set_title('Sentiment Analysis Results')

# Crear el gráfico de barras
bx = df.plot(kind='bar', x='Sentiment', y='Percentage')

# Agregar título al gráfico
bx.set_xlabel('Sentiment')
bx.set_ylabel('Percentage')
bx.set_title('Sentiment Analysis Results')

# Mostrar el gráfico
plt.show()